In [12]:
import pyvisa as visa
#import visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime

('ASRL1::INSTR',
 'ASRL4::INSTR',
 'GPIB0::1::INSTR',
 'GPIB0::3::INSTR',
 'GPIB0::6::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::8::INSTR',
 'GPIB0::13::INSTR',
 'GPIB0::23::INSTR')

In [ ]:

        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=20, skip_points=10, time_per_point=0.02):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 1        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
#     print(v_up)
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
#     print(v_down)
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v


def measure_resistance_on(dc, volt, dc_current=100e-6, measure_points=20, skip_points=10, time_per_point=0.02):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 1        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
#    print(v_up)
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
#    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
#    print(v_down)
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v

        
def trigger_ac(ac, offs=0.005, amp=1e-6, duration=0.005, wait_after_arm=2):
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(offs))
    ac.write("SOUR:WAVE:AMPL {}".format(amp))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(duration))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(wait_after_arm)
    ac.write("SOUR:WAVE:INIT")
    

    
def init_communication(amp=0.00005):
    print("check ac_1: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_1, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    
    print("check ac_2: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_2, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)
    """
    print("check ac_mid: offerset 50uA, amp1uA, duration 10s")
    trigger_ac(ac_mid, offs=amp, amp=1e-6, duration=5, wait_after_arm=2)
    time.sleep(15)"""
    
    dc.apply_current()# Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = amp            # Sets the source current to 0.1 mA
    dc.enable_source() 
    print("enable dc current now, current:" + str(dc))
    print("check nano volt:")
    for i in range(10):
        print(volt.query("fetch?"))
        time.sleep(0.1)
    time.sleep(10)
    dc.shutdown()   
    print("dc current disabled")

    
def do_measurement_three_pulses(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "C:/User Data/Hanshen Haung/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

def do_measurement_three_pulses_MOKE(sample_id='', ac_current=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None,count=1,time_step=0.1,wait_before_measure=0.1):
    file_name = "C:/User Data/Hanshen Haung/{}ac_{}uA{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current * 1e6), 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current, lockin1x, lockin1y, lockin1r, lockin1theta, lockin2x, lockin2y, lockin2r, lockin2theta\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    A=lock1.lock_in_measure(count=count, time_step=time_step, wait_before_measure=wait_before_measure)
    B=lock2.lock_in_measure(count=count, time_step=time_step, wait_before_measure=wait_before_measure)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current,A[0],A[1],A[2],A[3],B[0],B[1],B[2],B[3]))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                A=lock1.lock_in_measure(count=count, time_step=time_step, wait_before_measure=wait_before_measure)
                B=lock2.lock_in_measure(count=count, time_step=time_step, wait_before_measure=wait_before_measure)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current,A[0],A[1],A[2],A[3],B[0],B[1],B[2],B[3]))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""

    
def do_measurement_three_pulses_Asym(sample_id='', ac_current1=5*1e-3, ac_current2=5*1e-3, dc_current=100e-6, duration=0.005, round=6, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "C:/User Data/Hanshen Haung/{}_ac1_{}uA_ac2_{}uA_{}ms_wait{}s_{}.csv".format(sample_id, int(ac_current1 * 1e6), int(ac_current2 * 1e6),
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("round,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in range(round):
        print("round: " + str(i+1))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=ac_current1, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * ac_current1, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=ac_current1, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * ac_current1, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current2, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current2, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""





    
def do_measurement_one_pulse_FM(sample_id='', ac_current=[1*1e-3], dc_current=100e-6, duration=0.001,
                                wait_after_trigger=1, pulse=None):
    file_name = "C:/User Data/Hanshen Haung/{}ac_{}ms_wait{}s_{}.csv".format(sample_id, 
                                                                     int(duration * 1e3), wait_after_trigger, int(time.time()))
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("current,ac_source,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{}\n".format(0, 'init', average_v, std_v, dc_current))

    for i in ac_current:
        print("pulse current=:{} A".format(i))
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            if current == 'ac_1':
                    trigger_ac(ac_1, offs=i, duration=duration)
            elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * i, duration=duration)
            elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=i, duration=duration)
            elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * i, duration=duration)
            elif current == 'ac_2':
                    trigger_ac(ac_2, offs=i, duration=duration)
            elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * i, duration=duration)
            else:
                    print("no ac " + current)
                    raise
                    
            time.sleep(wait_after_trigger)
            average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
            with open(file_name, 'a') as f:
                f.write("{},{},{},{},{}\n".format(i, current, average_v, std_v, dc_current))
            print(average_v)
            time.sleep(0.1)
            
            
            
            
def do_measurement_three_pulses_currdep(sample_id='', ac_current=[5*1e-3], dc_current=100e-6, duration=0.005, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "C:/User Data/Hanshen Haung/{}_{}ms_wait{}s_{}.csv".format(sample_id, int(duration * 1e3), wait_after_trigger, int(time.time()))
    
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("ac_current,ac_source,point_count,average_v, std_v, dc_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in ac_current:
        print("ac_current: " + str(i*1e3) +" mA")
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=i, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * i, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=i, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * i, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=i, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * i, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""
                
def do_measurement_three_pulses_currdep_Asym(sample_id='', ac_current=[5*1e-3], ac_current2=5*1e-3, dc_current=100e-6, duration=0.005, point_per_round=5,
                                wait_after_trigger=15, pulse=None):
    file_name = "C:/User Data/Hanshen Haung/{}_{}ms_wait{}s_{}.csv".format(sample_id, int(duration * 1e3), wait_after_trigger, int(time.time()))
    
    if not pulse is None:
        file_name = file_name.replace(".csv", "".join(pulse) + ".csv")
    with open(file_name, 'w') as f:
        f.write("ac_current,ac_source,point_count,average_v, std_v, dc_current, reset_ac_current\n")
        
        
    average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
    with open(file_name, 'a') as f:
        f.write("{},{},{},{},{},{}\n".format(0, 'init', 0, average_v, std_v, dc_current))

    for i in ac_current:
        print("ac_current: " + str(i*1e3) +" mA")
        if pulse is None:
            pulse = ['ac_mid', '-ac_mid', 'ac_2', '-ac_2', 'ac_mid', '-ac_mid', 'ac_1', '-ac_1']
        for current in pulse:
            print("current:" + current)
            for point in range(point_per_round):
                if current == 'ac_1':
                    trigger_ac(ac_1, offs=i, duration=duration)
                elif current == '-ac_1':
                    trigger_ac(ac_1, offs=-1 * i, duration=duration)
                elif current == 'ac_mid':
                    trigger_ac(ac_mid, offs=i, duration=duration)
                elif current == '-ac_mid':
                    trigger_ac(ac_mid, offs=-1 * i, duration=duration)
                elif current == 'ac_2':
                    trigger_ac(ac_2, offs=ac_current2, duration=duration)
                elif current == '-ac_2':
                    trigger_ac(ac_2, offs=-1 * ac_current2, duration=duration)
                else:
                    print("no ac " + current)
                    raise
                    
                time.sleep(wait_after_trigger)
                average_v, std_v = measure_resistance(dc=dc, volt=volt, dc_current=dc_current)
                with open(file_name, 'a') as f:
                    f.write("{},{},{},{},{},{},{}\n".format(i, current, point, average_v, std_v, dc_current, ac_current2))
                print(average_v)
                time.sleep(3)
                """
                plt.clf()
                df = pd.read_csv(file_name)
                v = df['average_v']
                v.plot()
                plt.show()"""
                
                
rm = visa.ResourceManager()
rm.list_resources()




In [2]:
#Lock in
sensitivity = {1:26, 0.5:25, 0.2:24, 0.1:23, 0.05:22, 0.02:21, 0.01:20, 0.005:19, 0.002:18, 0.001:17, 0.0005:16, 0.0002:15,
               0.0001:14, 0.00005:13, 0.00002:12, 0.00001:11, 0.000005:10, 0.000002:9, 0.000001:8, 0.0000005:7, 0.0000002:6, 0.0000001:5, 0.00000005:4, 0.00000002:3,
               0.00000001:2, 0.000000005:1, 0.000000002:0}
               
class Lockin:
    def __init__(self, resource, gpib=1):
        self.lock_in = resource.open_resource('GPIB0::{}::INSTR'.format(gpib))
  #       self.lock_in.query('OUTX1;*IDN?')
  #      self.lock_in.write('*RST')
  #      self.lock_in.write('ALRM 0;KCLK 0;*ESE 53;ERRE 246;LIAE 7;*CLS;')

    def change_harmonic(self, harm=1, sens=1):
        self.lock_in.write("HARM {}".format(harm))
        self.lock_in.write("SENS {}".format(sensitivity[sens]))

    def lock_in_read(self):
        x = float(self.lock_in.query("OUTP? 1").strip())
        y = float(self.lock_in.query("OUTP? 2").strip())
        r = float(self.lock_in.query("OUTP? 3").strip())
        theta = float(self.lock_in.query("OUTP? 4").strip())
        return x, y, r, theta

    def lock_in_measure(self, count=10, time_step=0.1,
                        wait_before_measure=1,
                        harm=1, sens=0.00001):
#        self.change_harmonic(harm, sens)
        time.sleep(wait_before_measure)
        xs, ys, rs, thetas = [], [], [], []
        for i in range(count):
            x, y, r, theta = self.lock_in_read()
            xs += [x]
            ys += [y]
            rs += [r]
            thetas += [theta]
            time.sleep(time_step)
            
        xs = sorted(xs)
        ys=  sorted(ys)
        rs=  sorted(rs)
        thetas=sorted(thetas)
        
        if len(xs)>30:
            xs = xs[round(1/6*len(xs)): -round(1/6*len(xs))]
            ys = ys[round(1/6*len(ys)): -round(1/6*len(ys))]
            rs = rs[round(1/6*len(rs)): -round(1/6*len(rs))]
            thetas = thetas[round(1/6*len(thetas)): -round(1/6*len(thetas))]

        x_mean, x_std = np.mean(xs), np.std(xs)
        y_mean, y_std = np.mean(ys), np.std(ys)
        r_mean, r_std = np.mean(rs), np.std(rs)
        theta_mean, theta_std = np.mean(thetas), np.std(thetas)
        
        
        return x_mean, y_mean, r_mean, theta_mean, \
               x_std, y_std,r_std, theta_std

In [69]:
measure_resistance(dc=dc, volt=volt, dc_current=3e-4)


(0.000293651317125, 1.2789792676853964e-09)

In [4]:
#switch
ac_6221_1_id = 7
ac_6221__2id = 13

dc_2400_id = 23
nanovolt_id =3


ac_1 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221_1_id))
ac_2 = rm.open_resource('GPIB0::{}::INSTR'.format(ac_6221__2id))

dc = Keithley2400("GPIB0::{}".format(dc_2400_id))
volt = rm.open_resource('GPIB0::{}::INSTR'.format(nanovolt_id))


Kepco=rm.open_resource( 'GPIB0::2::INSTR')
lock1 = Lockin(rm, 1)
lock2 = Lockin(rm, 8)
Fluke=rm.open_resource( 'GPIB0::6::INSTR')
#FLUKE
Fluke.write('*')      #reset



3

In [29]:
#test lockin
lock1 = Lockin(rm, 1)
lock2 = Lockin(rm, 8)
A=lock1.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.0)
B=lock2.lock_in_measure(count=1, time_step=0.1, wait_before_measure=0.0)
print(A)
print(B)

(80.856, 1.43052e-06, 1.4782e-05, 1.4782e-05, 0.0, 0.0, 0.0, 0.0)
(2.54066e-06, -1.67639e-08, 2.53601e-06, -0.048, 0.0, 0.0, 0.0, 0.0)


In [75]:
currs=np.append(np.arange(10,25,1)*1e-3,np.arange(25,10,-1)*1e-3)
print(currs)

currs=[24*1e-3]
print(currs)



[0.01  0.011 0.012 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021
 0.022 0.023 0.024 0.025 0.024 0.023 0.022 0.021 0.02  0.019 0.018 0.017
 0.016 0.015 0.014 0.013 0.012 0.011]
[0.024]


In [56]:
for curr in currs:
    print('current is {}mA'.format(curr*1000))
    do_measurement_three_pulses(sample_id='LFO110_30nm_Pt5nm_longedgefirst_device2', ac_current=curr, dc_current=300e-6, duration=0.001, round=10, point_per_round=10,
                                wait_after_trigger=1, pulse=['ac_1','ac_2'])

current is 5.0mA
round: 1
current:ac_1
0.000289951724
0.00028996005987499995
0.0002899593835
0.00028995330800000004
0.0002899625555
0.000289985011625
0.00028996174525
0.00028995659374999997
0.000289961948875
0.00028996547750000004
current:ac_2
0.00028997632149999997
0.0002899602245
0.000289967185125
0.00028996938225
0.00028995891462499997
0.000289967292875
0.00028996494912500003
0.00028996096599999996
0.00028996511474999997
0.00028996557799999996
round: 2
current:ac_1
0.00028996110275000006
0.000289977264625
0.00028997901800000003
0.00028997077087499997
0.00028996528825
0.0002899818725
0.0002899688715
0.0002899710125
0.000289972541625
0.00028996903512499997
current:ac_2
0.00028996591575
0.00028997154675
0.000289970472
0.00028997793475
0.00028996910875
0.000289965537875
0.00028997252224999997
0.000289975111375
0.000289970441375
0.000289965704625
round: 3
current:ac_1
0.00028997190725
0.000289970827
0.000289981807125
0.00028997231837499997
0.000289967234375
0.000289975395375
0.0002899752

KeyboardInterrupt: 

In [76]:
for curr in currs:
    print('current is {}mA'.format(curr*1000))
    do_measurement_three_pulses_MOKE(sample_id='LFO110_30nm_Pt5nm_longedgefirst_device2', ac_current=curr, dc_current=300e-6, duration=0.001, round=20, point_per_round=10,
                                wait_after_trigger=1, pulse=['ac_1','ac_2'],count=5,time_step=0.1,wait_before_measure=0.1)

current is 24.0mA
round: 1
current:ac_1
0.000296226367375
0.000295728593625
0.000295346480375
0.00029535022225
0.000295215565875
0.00029517428
0.00029518307575
0.00029503089075
0.0002950104625
0.00029500304949999997
current:ac_2
0.0002968851155
0.00029690012074999996
0.000297200546875
0.0002973004535
0.00029736066925
0.000297650247625
0.00029762279324999997
0.00029768089074999995
0.00029766060287499997
0.00029775954062500004
round: 2
current:ac_1
0.000296494240625
0.00029596465012499996
0.000295726387625
0.000295707171
0.000295758367625
0.000295416005375
0.000295198435375
0.00029510125825000003
0.000294974262625
0.00029486055487500004
current:ac_2
0.00029651416225
0.000296881549125
0.000297026716
0.00029720490987499997
0.00029732281300000004
0.00029730572425
0.00029739151675000004
0.000297507584375
0.0002975662605
0.00029760796525
round: 3
current:ac_1
0.000296901543
0.00029625092625000006
0.000295918569
0.000295788177125
0.00029582630187499994
0.000295504550875
0.000295344264375
0.000

0.000298449738875
0.00029836338475000004
0.00029866375175000004


In [42]:
#be careful L1 *1e-3 or not 
print('6221 compliance 105V !!!!')1
L0=np.append(np.arange(10,40,10)*1e-3,np.arange(40,50,1)*1e-3)
L1=np.append(L0,np.arange(50,40,-1)*1e-3)
L1=np.append(np.arange(1,50,1)*1e-3,np.arange(50,0,-1)*1e-3)

print(L1)
print(len(L1))

6221 compliance 105V !!!!
[0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011 0.012
 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023 0.024
 0.025 0.026 0.027 0.028 0.029 0.03  0.031 0.032 0.033 0.034 0.035 0.036
 0.037 0.038 0.039 0.04  0.041 0.042 0.043 0.044 0.045 0.046 0.047 0.048
 0.049 0.05  0.049 0.048 0.047 0.046 0.045 0.044 0.043 0.042 0.041 0.04
 0.039 0.038 0.037 0.036 0.035 0.034 0.033 0.032 0.031 0.03  0.029 0.028
 0.027 0.026 0.025 0.024 0.023 0.022 0.021 0.02  0.019 0.018 0.017 0.016
 0.015 0.014 0.013 0.012 0.011 0.01  0.009 0.008 0.007 0.006 0.005 0.004
 0.003 0.002 0.001]
99


In [28]:
#be careful L1 *1e-3 or not 

do_measurement_three_pulses_currdep(sample_id='Fe2O3_30nm_Pt5nm_grad', ac_current=L1, dc_current=100e-6, duration=0.001, point_per_round=5,
                                wait_after_trigger=1, pulse=['ac_1','ac_2'])

ac_current: 10.0 mA
current:ac_1
1.7818850975e-05
1.78234297375e-05
1.78269228e-05
1.7843724199999995e-05
1.7829438262500003e-05
current:ac_2
1.7858713449999997e-05
1.75889865125e-05
1.7832798712500003e-05
1.7834403612500002e-05
1.781102475e-05
ac_current: 20.0 mA
current:ac_1
1.78521183125e-05
1.783336065e-05
1.7836239250000003e-05
1.7837926737499998e-05
1.781935425e-05
current:ac_2
1.7821642e-05
1.786206275e-05
1.78348136875e-05
1.78306157875e-05
1.78263623125e-05
ac_current: 30.0 mA
current:ac_1
1.7871102825e-05
1.7852336525e-05
1.7883977675e-05
1.7616626125e-05
1.786145855e-05
current:ac_2
1.78002818e-05
1.7786145862499998e-05
1.7823817575e-05
1.78171828875e-05
1.7800670399999998e-05
ac_current: 40.0 mA
current:ac_1
1.79401362875e-05
1.7917574850000004e-05
1.79303644625e-05
1.7939072225e-05
1.79130037875e-05
current:ac_2
1.7756542499999998e-05
1.7750290175e-05
1.7734698975e-05
1.7752753525e-05
1.7748862475e-05
ac_current: 41.0 mA
current:ac_1
1.79334158125e-05
1.796802425e-05
1.793

In [43]:
do_measurement_three_pulses_currdep_Asym(sample_id='Fe2O3_30nm_Pt5nm_grad_consRESETAC', ac_current=L1, ac_current2=48*1e-3,dc_current=100e-6, duration=0.001, point_per_round=10,
                                wait_after_trigger=1, pulse=['ac_1','ac_2'])

ac_current: 1.0 mA
current:ac_1
1.713885645e-05
1.71229164e-05
1.71153951875e-05
1.7109468587499998e-05
1.70997112625e-05
1.712236875e-05
1.7109543275e-05
1.70849308375e-05
1.70897384125e-05
1.71196365875e-05
current:ac_2
1.7188765425e-05
1.71932631625e-05
1.718091585e-05
1.7171550624999997e-05
1.7148778575e-05
1.715771875e-05
1.71564650375e-05
1.714592605e-05
1.71219676375e-05
1.71270680875e-05
ac_current: 2.0 mA
current:ac_1
1.7068524600000003e-05
1.70895562875e-05
1.7092024500000002e-05
1.7065571425e-05
1.70630821875e-05
1.70890560125e-05
1.7083629374999998e-05
1.7061208349999998e-05
1.70739011375e-05
1.708623655e-05
current:ac_2
1.7123893725e-05
1.71286496375e-05
1.7119312725e-05
1.71011346375e-05
1.7098989875e-05
1.70848195875e-05
1.710323955e-05
1.71022678375e-05
1.7070029575000002e-05
1.7086465475e-05
ac_current: 3.0 mA
current:ac_1
1.7053671024999998e-05
1.70331011875e-05
1.70650190625e-05
1.70334010375e-05
1.705816465e-05
1.705311915e-05
1.70535484e-05
1.7047093625e-05
1.70597

1.66121013875e-05
1.6609302e-05
1.65954564375e-05
1.6594288062499998e-05
1.65796824875e-05
1.65686375375e-05
1.6591388025e-05
1.6558541800000002e-05
1.66010345875e-05
1.6576064925e-05
ac_current: 21.0 mA
current:ac_1
1.66099010375e-05
1.6605579325e-05
1.66128965375e-05
1.6620124100000004e-05
1.6629205912500002e-05
1.66013887375e-05
1.66067849875e-05
1.66207757875e-05
1.6606471325e-05
1.65992074e-05
current:ac_2
1.6559038175e-05
1.6538801212499998e-05
1.65591113375e-05
1.658336365e-05
1.6562817912499998e-05
1.65755115e-05
1.65661642125e-05
1.6558317362499998e-05
1.656392445e-05
1.65329956125e-05
ac_current: 22.0 mA
current:ac_1
1.6612729462500003e-05
1.6592181737499998e-05
1.659051685e-05
1.66026998125e-05
1.65940615375e-05
1.65914312e-05
1.6599728262500003e-05
1.65751446875e-05
1.66087426e-05
1.6586376324999998e-05
current:ac_2
1.65654946625e-05
1.6540641425e-05
1.65404093625e-05
1.65513219375e-05
1.64454818875e-05
1.6544363325e-05
1.65469292e-05
1.65039620375e-05
1.6509394500000003e-0

1.6315814950000002e-05
1.629017865e-05
1.62864873e-05
1.6280634050000003e-05
1.631536015e-05
1.628302635e-05
current:ac_2
1.6115583675e-05
1.609163025e-05
1.6070884599999998e-05
1.61085064625e-05
1.6055147437500002e-05
1.6082309025e-05
1.6057722637500002e-05
1.60651556375e-05
1.6052038462499998e-05
1.60585635e-05
ac_current: 41.0 mA
current:ac_1
1.6291711625e-05
1.6281282275e-05
1.62834531375e-05
1.6312369325e-05
1.6297771825e-05
1.6287517325e-05
1.629694685e-05
1.62684866625e-05
1.62755277875e-05
1.6294823874999998e-05
current:ac_2
1.6067327324999998e-05
1.6069626312500003e-05
1.606309625e-05
1.6053244687500002e-05
1.60376274125e-05
1.6017634175e-05
1.6035080612499997e-05
1.6041983499999998e-05
1.6020279700000004e-05
1.60382455e-05
ac_current: 42.0 mA
current:ac_1
1.6303020337499998e-05
1.62637046875e-05
1.6291069875e-05
1.6271350612499998e-05
1.62616352625e-05
1.6261068825e-05
1.62658987e-05
1.62952991375e-05
1.62683210375e-05
1.6262797325e-05
current:ac_2
1.60436666375e-05
1.5905294

1.66432667125e-05
1.66658595375e-05
1.667522655e-05
1.66567946e-05
1.66624803e-05
1.6676516175e-05
1.66357440375e-05
1.666705825e-05
1.66639968e-05
current:ac_2
1.6443372825e-05
1.6409623375e-05
1.6421980737499998e-05
1.6420781225e-05
1.6433970187500003e-05
1.6418959349999998e-05
1.64316021125e-05
1.6396572387499997e-05
1.6391061e-05
1.638046525e-05
ac_current: 39.0 mA
current:ac_1
1.66363294375e-05
1.663713125e-05
1.6625649675e-05
1.66548168375e-05
1.6631710200000003e-05
1.66535775625e-05
1.66322580125e-05
1.6625494849999998e-05
1.661724575e-05
1.66183340375e-05
current:ac_2
1.64324623125e-05
1.6411862625e-05
1.64311450875e-05
1.64079114125e-05
1.6398615824999998e-05
1.6392727512500002e-05
1.64117490375e-05
1.6387394175e-05
1.6377835275e-05
1.63799499125e-05
ac_current: 38.0 mA
current:ac_1
1.6551932e-05
1.65916533e-05
1.6583634937499998e-05
1.65807725125e-05
1.65715780375e-05
1.6563537725000002e-05
1.656365435e-05
1.6567807224999998e-05
1.6525215575e-05
1.6579475562500002e-05
current

1.6052133537500002e-05
1.6055866100000002e-05
ac_current: 20.0 mA
current:ac_1
1.61162142875e-05
1.61184563e-05
1.610987335e-05
1.61035709e-05
1.61108270125e-05
1.6094924975e-05
1.6093290225e-05
1.61077824625e-05
1.61184779e-05
1.6088914425e-05
current:ac_2
1.6078156349999998e-05
1.6093905075e-05
1.6065785575e-05
1.60484199e-05
1.60350686e-05
1.60513777e-05
1.6043815737499998e-05
1.60383150625e-05
1.60391783375e-05
1.60095157875e-05
ac_current: 19.0 mA
current:ac_1
1.606635785e-05
1.60448751125e-05
1.6046258762500002e-05
1.6037816575000003e-05
1.6067488625e-05
1.60716506375e-05
1.6059598275e-05
1.60646601e-05
1.60508537375e-05
1.6052701312499998e-05
current:ac_2
1.6028574825e-05
1.6018534987499998e-05
1.59962886875e-05
1.6015528025e-05
1.6018163749999998e-05
1.6040118275e-05
1.60042425625e-05
1.6010959762500002e-05
1.6042945249999997e-05
1.60423451375e-05
ac_current: 18.000000000000004 mA
current:ac_1
1.6037564924999998e-05
1.6020253325e-05
1.60200186875e-05
1.60398891875e-05
1.6038909

1.57489212875e-05
1.573146065e-05
1.57513118125e-05
1.572362175e-05
1.57689550125e-05
1.57039996125e-05
1.574391315e-05
1.5724241375e-05
1.57113459e-05
1.57216325375e-05


In [ ]:
LVread()

In [ ]:
ac_1.write("SOUR:WAVE:ABOR")


In [5]:
def setcurr(curr=1, count_write=50, thrd=0.01):    
    for i in range(count_write):
        
            magnet.write("CURR {}".format(curr))
            time.sleep(0.1)
    print ('Set current done.', curr)

def covert_to_float(s):
    ans = ''
    for i in range(len(s) - 2):
        ans += s[i]
        if s[i+1].isalpha() & s[i+2].isalpha(): break
    return float(ans)

def fieldmeasure():
    s = gauss.query(":FETCH?")
    field=covert_to_float(s)*10000
    return field

def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

In [6]:
def nano_change_chanel(volt, chan=None):
    if not chan:
        return
    volt.write(":SENS:CHAN {}".format(chan))
    time.sleep(0.1)
    if volt.query(":SENS:CHAN?").strip() == str(chan):
        return
    else:
        print("channel switch error")
        raise
        
def measure_resistance_two_channel(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    
    dc.compliance_voltage = 100        # Sets the compliance voltage to 100 V
    
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    dc.shutdown() 

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2


def field_scan_one_sample_two_channel(dc, volt, dc_current_1=1e-4, currs=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        
        dc_current = None
        offset = fieldmeasure()
        print ('offset is', offset)
        for curr in currs:
            setcurr(curr)
            field = fieldmeasure() - offset
            time.sleep(field_wait)
            print ('field is', field)
            for file in [file_1]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc, volt=volt, dc_current=dc_current)
                   
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

In [7]:
def measure_resistance_two_channel_6221(ac, volt, ac_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current

    
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(ac_current))
    ac.write("SOUR:WAVE:AMPL {}".format(2e-7))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(100))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(2)
    ac.write("SOUR:WAVE:INIT")
    
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    #ac.current = -1 * ac_current
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(-ac_current))
    ac.write("SOUR:WAVE:AMPL {}".format(2e-7))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(100))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(2)
    ac.write("SOUR:WAVE:INIT")
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    ac.write("SOUR:WAVE:ABOR")

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2



def measure_resistance_one_channel_6221(ac, volt, ac_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current

    
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(ac_current))
    ac.write("SOUR:WAVE:AMPL {}".format(2e-7))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(100))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(2)
    ac.write("SOUR:WAVE:INIT")
    
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    

    
    # Sets the compliance voltage to 10 V
    #ac.current = -1 * ac_current
    ac.write("SOUR:WAVE:ABOR")
    ac.write("SOUR:WAVE:OFFS {}".format(-ac_current))
    ac.write("SOUR:WAVE:AMPL {}".format(2e-7))
    ac.write("SOUR:WAVE:DUR:TIME {}".format(100))
    ac.write("SOUR:WAVE:ARM")
    time.sleep(2)
    ac.write("SOUR:WAVE:INIT")
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        

    
    ac.write("SOUR:WAVE:ABOR")

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]

    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1



def field_scan_one_sample_two_channel_6221(ac, volt, ac_current_1=1e-4, currs=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(ac_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        
        ac_current = None
        offset = fieldmeasure()
        print ('offset is', offset)
        for curr in currs:
            setcurr(curr)
            field = fieldmeasure() - offset
            time.sleep(field_wait)
            print ('field is', field)
            for file in [file_1]:
                if file == file_1:
                    ac_current = ac_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel_6221(ac=ac_1, volt=volt, ac_current=ac_current)
                   
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, ac_current))
                    
                    
def no_field_scan_one_sample_two_channel_6221(ac, volt, ac_current_1=1e-4, Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(ac_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        
        ac_current = None

        for file in [file_1]:
                if file == file_1:
                    ac_current = ac_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel_6221(ac=ac_1, volt=volt, ac_current=ac_current)
                   
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, ac_current))
                    


In [ ]:
setcurr(curr=0)

In [ ]:
! pip install -U pyvisa

In [ ]:
visa.__version__

In [ ]:
magnet.write("FUNC:MODE CURR")

In [ ]:
magnet.write("CURR:RANG 1")

In [ ]:
magnet.write("VOLT:RANG 1")

In [ ]:
magnet.write("OUTPUT ON;CURR 0")

In [ ]:
magnet.write("CURR 0")

In [ ]:
fs = [18]
steps = [3]
currs = generate_fields(fs, steps)
print(currs)


In [ ]:
field_scan_one_sample_two_channel(dc, volt, dc_current_1=1e-4, currs=currs, sample1_id='#3111_30nmFe2O3_4nmMo_10nmWTe2', sample2_id="2",
                                      t_wait=1, field_wait=1)
#setcurr(curr=0)

In [ ]:
no_field_scan_one_sample_two_channel_6221(ac_1, volt, ac_current_1=2e-4, sample1_id='test1', sample2_id="2",
                                      t_wait=1, field_wait=1)

In [ ]:
field_scan_one_sample_two_channel_6221(ac_1, volt, ac_current_1=2e-4, currs=currs, sample1_id='test1', sample2_id="2",
                                      t_wait=1, field_wait=1)
setcurr(curr=0)

In [ ]:
class Magnet:
    def __init__(self, resource, gpib=6):
        self.magnet = resource.open_resource('GPIB0::{}::INSTR'.format(gpib))

    def Open(self):
        self.magnet.write("FUNC:MODE CURR")
        self.magnet.write("CURR:RANG 1")
        self.magnet.write("VOLT:RANG 1")
        self.magnet.write("OUTPUT ON;CURR 0")
        self.magnet.write("VOLT 20")
        
    def SetCurr(self, curr=1, count_write=30, thrd=0.01):
        
        for i in range(count_write):
                self.magnet.write("CURR {}.format(curr)")
                time.sleep(0.5)
        print ('Set current done.', i)
        
        for i in range(count_write):
                self.magnet.write("MEAS:CURR?")
                time.sleep(0.5)
        print ('Measure write done.', i)
            
        not_succeeded, run_count, x = True, 0, float('inf')
        while not_succeeded or (abs(x - curr) > thrd):
            try:
                x = float(self.magnet.read("MEAS:CURR?"))
                not_succeeded = False
                print ('Measured current is', x)
            except Exception as e:
                run_count += 1
                print ('run count:', run_count)

        return x


In [ ]:
rm = visa.ResourceManager()
mag = Magnet(rm, 6)

In [ ]:
mag.Open()

In [ ]:
for i in range(60):
            magnet.write("CURR {}".format(3))
            time.sleep(0.05)
print ('Set current done.', i)





In [ ]:
with  rm.open_resource(rm.list_resources()[0]) as test:
    # Start by clearing the device, which should discard the output device of the buffer
    test.clear()

In [ ]:
mag.magnet.write("MEAS:CURR?")

In [ ]:
for i in range(50):
            mag.magnet.write("MEAS:CURR?")
            time.sleep(0.05)
            print ('Set current done.', i)


In [ ]:
mag.magnet.read("MEAS:CURR?")

In [ ]:
with  rm.open_resource(rm.list_resources()[0]) as test:
    test.flush(visa.constants.VI_READ_BUF)
    test.flush(visa.constants.VI_WRITE_BUF_DISCARD)
    test.clear()

In [ ]:
visa.constants.StatusCode

In [ ]:
mag.magnet.read_raw()

In [ ]:
from pyvisa.constants import StopBits
mag.magnet.stop_bits = StopBits.one

In [ ]:
magnet.write("*RST")


In [ ]:
magnet.write("MEAS:CURR? ")

In [ ]:
magnet.read("MEAS:CURR?")

In [ ]:
magnet.write("*IDN?")


In [ ]:
magnet.read("*IDN?")

In [ ]:
mag.magnet.read_bytes(1000, break_on_termchar='\n')

In [ ]:
magnet.write("FUNC:MODE CURR")
magnet.write("CURR:RANG 1")
magnet.write("VOLT:RANG 1")
magnet.write("OUTPUT ON;CURR 0")
magnet.write("VOLT 20")

In [ ]:
magnet.write("CURR 0")

In [ ]:
init_communication(amp=1e-4)

In [ ]:
do_task = True
check_conn = False
anneal_at_high_T = False
anneal_current = 17*1e-3

In [ ]:

if not do_task:
    raise

t_array = [300]



i = [26*1e-3, 28*1e-3]

pulse_durations = [0.001]
fields = [0]
wait_after_T = 0.1 # in minute




if check_conn:
    do_measurement_three_pulses(sample_id='LMB214_15nm_3nm__8leg_0dsegree_0T_switch_10sec_3sec__Noanneal'+ "G",
                   ac_current=10*1e-3, dc_current=10e-5, duration=0.001, 
                   round=2, point_per_round=4, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])
    raise

if anneal_at_high_T:
#    set_T(350)
    do_measurement_three_pulses(sample_id='LMB214_15nm_3nm__8leg_0dsegree_0T_switch_10sec_3sec__Noanneal'+ "G",
               ac_current=anneal_current, dc_current=10e-5, duration=0.001, 
               round=4, point_per_round=10, wait_after_trigger=10, pulse=['ac_1', 'ac_2'])

    
for t in t_array:
    print(datetime.datetime.now())
    print("set T to:" + str(t))
    print(datetime.datetime.now())
    print("start measurment:")
    if not t is None:
#        set_T(t)
        time.sleep(wait_after_T*60)
    
    for c in i:
        for field in fields:
            for pulse_d in pulse_durations:
                print("current pulse:" + str(c))
                print("set field to:" + str(field))
                
                print(datetime.datetime.now())
                do_measurement_three_pulses(sample_id='test3'+ str(t) +"K" + str(field) + "G",
                               ac_current=c, dc_current=10e-5, duration=pulse_d, 
                               round=2, point_per_round=5, wait_after_trigger=5, pulse=['ac_1', 'ac_2'])

do_task = False

In [ ]:
from labview_automation import LabVIEW